In [2]:
#Procedimiento realizado a la tabla precios_semanas_20200419_20200426.xlsx
#  especificamente precios_20200426
#   1.Se borran los datos repetidos si existen y los valores null

import pandas as pd
import xlrd
df_ps_20200426 = pd.read_excel('Datasets\precios_semanas_20200419_20200426.xlsx ', 'precios_20200426_20200426')

#-------------Tratamiento General
# Se borran valores repetidos
df_ps_20200426.drop_duplicates(inplace=True)

#Se borran valores nulos
df_ps_20200426.dropna(inplace=True)

#-----------Manejo de los Id del columna sucursal_id
#Se convierte el codigo string en una lista con 
# los subcodigos (comercioId-banderaId-sucursalId.) 
Ids =[]
for id_Tex in df_ps_20200426["sucursal_id"]:
    Ids.append(id_Tex.split("-")) 
    
#Se procede a crear la nueva columna con los nuevos Id_Sucursal
Id_Sucursal =[]
for Idcompuesta in Ids:
    num = int(Idcompuesta[0])*10000*100  +  int(Idcompuesta[1])*10000 + int(Idcompuesta[2])
    Id_Sucursal.append(num)
    
#Se inserta la nueva columna y se borra la anterior
df_ps_20200426.insert(0,"Id_Sucursal", Id_Sucursal, allow_duplicates=False)
df_ps_20200426.drop("sucursal_id",axis=1,inplace=True)

#------------Manejo de los Id del columna producto_id
#Se agrega la columna ID_producto con valores de 'producto_id'
df_ps_20200426.insert(loc = 0 , column = "Id_producto", value = df_ps_20200426['producto_id'])

#Se elimina la columna producto_id
df_ps_20200426.drop(["producto_id"], axis=1,inplace=True)

#Se exporta la base de datos a MySQL
import sqlalchemy as db
database_username='root'            # Nombre de cliente en MySQL Workbrench
database_password='JCSR.MySQL123'   # Contraseña de MySQL Workbrench
database_ip='localhost'
database_name='ETL_Lab01'           # Nombre de Base de datos a donde nos conectaremos (se debio crear primero en Mysql)
database_conection = db.create_engine(f'mysql+pymysql://{database_username}:{database_password}@{database_ip}/{database_name}',
                                    encoding='latin1')
coneccion=database_conection.connect()
df_ps_20200426.to_sql('ps_20200426',coneccion, index=False)

#Creacion de conexion con Mysql 
import pymysql
conexion = pymysql.connect(
   host = 'localhost',
   user = 'root',
   passwd='JCSR.MySQL123',
   db='ETL_Lab01'           #LA base de datos se debio crear primero en Mysql
)
cursor = conexion.cursor()
try:
    #Se crean las relaciones
    cursor.execute("SET FOREIGN_KEY_CHECKS=0;")
    cursor.execute("alter table ps_20200426 add foreign key (Id_Sucursal) references sucursal(Id_Sucursal);")
    cursor.execute("alter table ps_20200426 add foreign key (Id_producto) references producto(Id_producto);")
    cursor.execute("SET FOREIGN_KEY_CHECKS=1;")
finally:
    conexion.close()
print("Carga de tabla 'ps_20200426' a MySQL completa")

Carga de tabla 'ps_20200426' a MySQL completa


In [9]:
#------------Manejo de los Id del columna producto_id

#Falta solucionar un tema de formato no es complejo pero hay que rev 
# desde la tabla producto y todas las otras

#Se verifica el producto_id creando lista para la columna con valores Id 
# que se puedan convertir en int 
id_producto =[]
for i in range(len(df_ps_20200426)):
    try:
        id_producto.append(int(df_ps_20200426.iloc[i,2]))
    except ValueError:
        id_producto.append(None)
        
#Se agrega la columna ID_producto con valores de id int
df_ps_20200426.insert(loc = 0 , column = "Id_producto", value = id_producto)

#Se guardan valores con problemas, con mas tiempo se pueden mejorar e integrar a la tabla
df_ps_20200426_erroresId = df_ps_20200426[df_ps_20200426["Id_producto"].isnull()].drop(["Id_producto"], axis=1)

#Se borran valores nulos o con problemas
df_ps_20200426.dropna(subset="Id_producto",inplace=True)

#Se elimina la columna producto_id
df_ps_20200426.drop(["producto_id"], axis=1,inplace=True)